In [4]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import texthero as hero
from texthero import preprocessing
sb.set()

In [5]:
steamData = pd.read_csv('steam_data/steam.csv')
steamSupportInfo = pd.read_csv('steam_data/steam_support_info.csv')
steamChart = pd.read_csv('steam_data/steamCharts.csv')
steamSpyTagData = pd.read_csv('steam_data/steamspy_tag_data.csv')
steamGames = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_games/steam_games*.csv"))))
steamMediaData = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_media_data/steam_media_data*.csv"))))
steamRequirements = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_requirements/steam_requirements*.csv"))))
steamDescription = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_description/steam_description*.csv"))))

#game reviews
steam_HalfOfTheGameReviews = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/steam_review_*.csv"))))
files = glob.glob("steam_data/steam_reviews" + '**/**' + "/*.csv")
steamReviews_fromFolder = pd.concat([pd.read_csv(f) for f in files ])
steamReviewsDF = [steam_HalfOfTheGameReviews, steamReviews_fromFolder]
steamCombinedReviews = pd.concat(steamReviewsDF)

C:\Education\Study\EE0005\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Education\Study\EE0005\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Education\Study\EE0005\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Education\Study\EE0005\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
#rename for easier identification
steamGames = steamGames.rename(columns={"name": "title"})

#create a custom cleaning pipeline
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_whitespace]

steamCombinedReviews['title'] = hero.clean(steamCombinedReviews['title'], pipeline = custom_pipeline)
steamGames['title'] = hero.clean(steamGames['title'], pipeline = custom_pipeline)

Find average time needed to complete a game with regards to how popular it is.

Find price of game with regards to number of players



In [7]:
#cleaning steam reviews dataset, removing reviews of 0 playtime
steamCombinedReviews_clean = steamCombinedReviews.copy()

steamCombinedReviews_clean["author.playtime_at_review"].fillna(value="")

0        586.0
1       5269.0
2       7993.0
3       6794.0
4       6504.0
        ...   
386    21594.0
387     9443.0
388     4736.0
389    31836.0
390    15128.0
Name: author.playtime_at_review, Length: 4532560, dtype: object

In [8]:
steamGames_price = steamGames[['title', 'original_price']]
steamCombinedReviews_Title = steamCombinedReviews[['title']]
matchTitle1 = steamGames_price.title.isin(steamCombinedReviews_Title.title)
steamMatchTitle1 = steamGames_price[matchTitle1]

steamUniqueTitle1 = steamMatchTitle1.drop_duplicates(['title'])
steamUniqueTitle1 = steamUniqueTitle1.reset_index(drop=True)

steamUniqueTitle1
nan_value = float("NaN")
steamUniqueTitle1.replace("", nan_value, inplace=True)
steamUniqueTitle1


,title,original_price
0,playerunknown s battlegrounds,$29.99
1,battletech,$39.99
2,dayz,$44.99
3,human fall flat,$14.99
4,they are billions,$29.99
...,...,...
135,atlas,$29.99
136,metro exodus,$24.99
137,transport fever 2,$38.99
138,halo the master chief collection,$49.99


In [9]:
#plot y(no. of review) against x(price)

#count number of reviews for each game (clean data)


steamGames_titles = steamGames[['title']]
steamCombinedReviews_Title = steamCombinedReviews[['title']]
matchTitle2 = steamGames_titles.title.isin(steamCombinedReviews_Title.title)
steamMatchTitle2 = steamGames_titles[matchTitle2]


In [10]:

steamData_playtime = steamData[['name', 'average_playtime']]
steamData_playtime
print("Number of times a user did not play a game they owned:", 
      len(steamData_playtime[steamData_playtime['average_playtime'] == 0]))
#shows that many players do not actually even start a game that they actually own


Number of times a user did not play a game they owned: 20905


In [40]:
steamData_playtime[steamData_playtime!= 0]
steamData_playtime.replace({'0':np.nan, 0:np.nan})
steamData_playtime.dropna(subset = ["average_playtime"], inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27075 entries, 0 to 27074
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              27075 non-null  object
 1   average_playtime  27075 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 634.6+ KB


<ipython-input-40-279b3cbf9f43>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steamData_playtime.dropna(subset = ["average_playtime"], inplace=True)


,name,average_playtime
0,Counter-Strike,17612
1,Team Fortress Classic,277
2,Day of Defeat,187
3,Deathmatch Classic,258
4,Half-Life: Opposing Force,624
...,...,...
27070,Room of Pandora,0
27071,Cyber Gun,0
27072,Super Star Blast,0
27073,New Yankee 7: Deer Hunters,0


In [ ]:
#sorting games by average minutes of playtime per game when reviewing